# Final Project - Project Proposal

## Problem Statement

## Key Elements ##
- The target variable you want to predict.
- How predicting that target variable could help with some kind of decision.
- The features you want to use to predict that target variable.
- Goals and success metrics
- Risks or limitations
- Identify at least one relevant dataset and load it into a Pandas dataframe.

**Predicted Variable**
I want to predict the location of care seeking for a patient with certain symptoms/disease

**Decision Support**
In order to meet patients where they access care, health systems need to better understand patterns of care seeking that patients exhibit. 

**Features**
I would want to predict patient care seeking based on geography, education level, income level, distance to different health facilities, size of their home city, proximity of their home city to a big city. 

**Dataset** I'll plan to use the Demographic and Health Survey - a national level survey that is done in countries around the world focusing on demographic and health related questions. 